## Feature: Batting Strength

A measure of a team's batting capability based on player statistics.

Approach:

Calculate the sum of the strike rates and average runs per match of all batters in a team.

Normalize this value for consistency.

In [ ]:
# Calculate individual batter strike rates
batsman_stats = deliveries.groupby("batter").agg({"batsman_runs": "sum", "ball": "count"})
batsman_stats["strike_rate"] = (batsman_stats["batsman_runs"] / batsman_stats["ball"]) * 100

# Aggregate batting strength per team
batting_strength = deliveries.groupby("batting_team")["batter"].apply(lambda x: x.map(batsman_stats["strike_rate"]).sum())
deliveries["batting_strength"] = deliveries["batting_team"].map(batting_strength)
